In [2]:
import os
import json
import pandas as pd
import traceback

In [26]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.9,<4.0; 0.0.10rc0 Requires-Python >=3.9,<4.0; 0.0.11 Requires-Python >=3.9,<4.0; 0.0.1rc0 Requires-Python >=3.9,<4.0; 0.0.2 Requires-Python >=3.9,<4.0; 0.0.3 Requires-Python >=3.9,<4.0; 0.0.4 Requires-Python >=3.9,<4.0; 0.0.5 Requires-Python >=3.9,<4.0; 0.0.6 Requires-Python >=3.9,<4.0; 0.0.7 Requires-Python >=3.9,<4.0; 0.0.8 Requires-Python >=3.9,<4.0; 0.0.9 Requires-Python >=3.9,<4.0; 1.0.1 Requires-Python <4.0,>=3.9; 1.0.2 Requires-Python <4.0,>=3.9; 1.0.3 Requires-Python <4.0,>=3.9; 1.0.4 Requires-Python <4.0,>=3.9; 1.0.5 Requires-Python <4.0,>=3.9; 1.0.6 Requires-Python <4.0,>=3.9; 1.0.7 Requires-Python <4.0,>=3.9
ERROR: Could not find a version that satisfies the requirement langchain-google-genai (from versions: none)
ERROR: No matching distribution found for langchain-google-genai


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\McQue\Desktop\DeepLearning\GENAI\langchain-crash-course\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [8]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002C712E5B790>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002C712E56A50>, default_metadata=())

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
#import PyPDF2


In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [13]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [15]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\McQue\Desktop\DeepLearning\GENAI\langchain-crash-course\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [26]:
file_path=r"C:\Users\McQue\Desktop\DeepLearning\GENAI\langchain-crash-course\mcq_gen\data.txt"

In [27]:
file_path

'C:\\Users\\McQue\\Desktop\\DeepLearning\\GENAI\\langchain-crash-course\\mcq_gen\\data.txt'

In [37]:
with open(file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()


In [38]:
print(TEXT)

Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.

How does machine learning work?
UC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.
An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.
A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and the model estimate

In [39]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"


In [41]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\McQue\Desktop\DeepLearning\GENAI\langchain-crash-course\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.

How does machine learning work?
UC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.
An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.
A Model Optimization Process: If the model can fit better to the data points in the traini

In [42]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:8245
Prompt Tokens:7280
Completion Tokens:965
Total Cost:0.0


In [43]:
response

{'text': 'Machine learning (ML) is a branch of artificial intelligence (AI) and computer science that focuses on the using data and algorithms to enable AI to imitate the way that humans learn, gradually improving its accuracy.\n\nHow does machine learning work?\nUC Berkeley (link resides outside ibm.com) breaks out the learning system of a machine learning algorithm into three main parts.\n\nA Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.\nAn Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.\nA Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and t

In [44]:
quiz=response.get("quiz")

In [57]:
print(quiz)

```json
{
"1": {
"mcq": "Which of the following is NOT a part of a machine learning algorithm's learning system?",
"options": {
"a": "Decision Process",
"b": "Error Function",
"c": "Model Optimization Process",
"d": "Data Cleaning Process"
},
"correct": "d"
},
"2": {
"mcq": "What is the key difference between deep learning and classical machine learning?",
"options": {
"a": "Deep learning requires labeled data, while classical machine learning uses unlabeled data.",
"b": "Deep learning can learn from unstructured data, while classical machine learning requires more human intervention and structured data.",
"c": "Deep learning uses error functions, while classical machine learning does not.",
"d": "Deep learning is used for image recognition, while classical machine learning is used for fraud detection."
},
"correct": "b"
},
"3": {
"mcq": "Which type of machine learning uses a smaller labeled dataset to guide classification and feature extraction from a larger, unlabeled dataset?",
"opt

In [67]:
quiz = quiz.strip('```')
quiz = quiz.strip('json')

In [68]:
print(quiz)


{
"1": {
"mcq": "Which of the following is NOT a part of a machine learning algorithm's learning system?",
"options": {
"a": "Decision Process",
"b": "Error Function",
"c": "Model Optimization Process",
"d": "Data Cleaning Process"
},
"correct": "d"
},
"2": {
"mcq": "What is the key difference between deep learning and classical machine learning?",
"options": {
"a": "Deep learning requires labeled data, while classical machine learning uses unlabeled data.",
"b": "Deep learning can learn from unstructured data, while classical machine learning requires more human intervention and structured data.",
"c": "Deep learning uses error functions, while classical machine learning does not.",
"d": "Deep learning is used for image recognition, while classical machine learning is used for fraud detection."
},
"correct": "b"
},
"3": {
"mcq": "Which type of machine learning uses a smaller labeled dataset to guide classification and feature extraction from a larger, unlabeled dataset?",
"options": 

In [69]:
quiz=json.loads(quiz)

In [71]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [72]:
quiz_table_data

[{'MCQ': "Which of the following is NOT a part of a machine learning algorithm's learning system?",
  'Choices': 'a: Decision Process | b: Error Function | c: Model Optimization Process | d: Data Cleaning Process',
  'Correct': 'd'},
 {'MCQ': 'What is the key difference between deep learning and classical machine learning?',
  'Choices': 'a: Deep learning requires labeled data, while classical machine learning uses unlabeled data. | b: Deep learning can learn from unstructured data, while classical machine learning requires more human intervention and structured data. | c: Deep learning uses error functions, while classical machine learning does not. | d: Deep learning is used for image recognition, while classical machine learning is used for fraud detection.',
  'Correct': 'b'},
 {'MCQ': 'Which type of machine learning uses a smaller labeled dataset to guide classification and feature extraction from a larger, unlabeled dataset?',
  'Choices': 'a: Supervised learning | b: Unsupervise

In [73]:
quiz=pd.DataFrame(quiz_table_data)

In [74]:
quiz.to_csv("machinelearning.csv",index=False)

In [75]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_30_2024_11_51_45'

In [5]:
from datetime import datetime

LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%M_%M_%S')}.log"
print(LOG_FILE)

07_02_2024_22_22_19.log
